### Sources

http://billchambers.me/tutorials/2015/01/14/python-nlp-cheatsheet-nltk-scikit-learn.html

http://glowingpython.blogspot.de/2013/07/combining-scikit-learn-and-ntlk.html

http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html

https://stackoverflow.com/questions/10526579/use-scikit-learn-to-classify-into-multiple-categories

https://github.com/anuraagvak/IRE-PersonalityRecognition-Final/blob/master/ire_report.pdf

https://github.com/Charudatt89/Personality_Recognition/blob/master/22-9-PersonalityRecognition/Report/Report.pdf

http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python

Lexical and syntactic features: http://www.aicbt.com/authorship-attribution/

In [65]:
from IPython.core.display import HTML
HTML("<style>.container { width:75% !important; }</style>")
# HTML("<style>div.cell.selected {border-left-width: 1px !important;}</style>")

In [58]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *
from rpy2 import *

ran.seed(5125)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.grid_search import *
from sklearn.linear_model import *
from sklearn.cross_validation import *
# n.download()

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### Read data and show them

In [59]:
data = pd.read_csv("../raw_data/data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=2)
# data.describe()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [60]:
d = {'n': 0, 'y': 1}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

### Split data into train + test


In [61]:
train_feat_neu, test_feat_neu, train_class_neu, test_class_neu = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify = neu["cNEU"],random_state= 5152)
print(train_feat_neu.shape, test_feat_neu.shape,train_class_neu.shape ,test_class_neu.shape)
# print(train_class_neu)
# print(train_feat_neu.iloc[:,1])

neu_X = train_feat_neu.append(test_feat_neu)
neu_Y = train_class_neu.append(test_class_neu)

train_feat_ext, test_feat_ext, train_class_ext, test_class_ext = sk.cross_validation.train_test_split(ext, ext["cEXT"], train_size = 0.66, stratify = ext["cEXT"],random_state= 5152)

ext_X = train_feat_ext.append(test_feat_ext)
ext_Y = train_class_ext.append(test_class_ext)

train_feat_agr, test_feat_agr, train_class_agr, test_class_agr = sk.cross_validation.train_test_split(agr, agr["cAGR"], train_size = 0.66, stratify = agr["cAGR"],random_state= 5152)

agr_X = train_feat_agr.append(test_feat_agr)
agr_Y = train_class_agr.append(test_class_agr)

train_feat_con, test_feat_con, train_class_con, test_class_con = sk.cross_validation.train_test_split(con, con["cCON"], train_size = 0.66, stratify = con["cCON"],random_state= 5152)

con_X = train_feat_con.append(test_feat_con)
con_Y = train_class_con.append(test_class_con)

train_feat_opn, test_feat_opn, train_class_opn, test_class_opn = sk.cross_validation.train_test_split(opn, opn["cOPN"], train_size = 0.66, stratify = opn["cOPN"],random_state= 5152)

opn_X = train_feat_opn.append(test_feat_opn)
opn_Y = train_class_opn.append(test_class_opn)

(6545, 3) (3372, 3) (6545,) (3372,)


### Creates diff. pipelines here!

with stop_words : ‘english’ in CountVecto or T.Vecto accuracy ussualy lower

In [62]:
target_names = ['yes', 'no']

# 3 Support Vector Machines
classifierLinearSVC = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC(random_state = 5152))]) # OneVsRestClassifier already implemented, docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification

classifierSVC = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('clf', SVC(cache_size=500, random_state = 5152, kernel = "rbf", decision_function_shape = "ovr"))]) # "one-against-one", docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification

# 2 Naive Bayes
classifierNB = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('nb', MultinomialNB())])

classifierBNB = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('bnb', BernoulliNB())])


# Nearest Neighboars
classifier_kNN = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('kNN', KNeighborsClassifier(n_jobs=-1))]) # http://scikit-learn.org/stable/modules/neighbors.html#neighbors

# 2 Ensamble methods
classifierRF = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('rfc', RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state = 5152))])

classifierADC = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('adc', AdaBoostClassifier(random_state = 5152))])


#### Grid Search

In [63]:
# parametersSVC = {
#     'tfidf__max_df': (0.5, 0.75, 1.0),
#     #'tfidf__max_features': (None, 5000, 10000, 50000),
#     'tfidf__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#     'tfidf__use_idf': (True, False),
#     'tfidf__norm': ('l1', 'l2', None),
#     'tfidf__smooth_idf': (True, False),
#     #'clf__loss': ("hinge", "squared_hinge"),
#     #'clf__penalty': ('l1', 'l2'),
#     'clf__C': (0.5, 0.75, 1.0),
#     'clf__dual': (True, False),
#     'tfidf__stop_words' : ("english", None),
#     #'tfidf__lowercase' : (True, False),
#     'clf__multi_class': ('ovr', 'crammer_singer')
# }

In [64]:
# Display progress logs on stdout
# logging.basicConfig(level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(message)s')

# grid_search = GridSearchCV(classifierLinearSVC, parametersSVC, n_jobs=-1, verbose=1)

# print("Performing grid search...")
# print("pipeline:", [name for name, _ in classifierLinearSVC.steps])
# print("parameters:")
# pprint(parametersSVC)
# t0 = time()
# grid_search.fit(neu["STATUS"], neu["cNEU"])
# print("done in %0.3fs" % (time() - t0))
# print()

# print("Best score: %0.3f" % grid_search.best_score_)
# print("Best parameters set:")
# best_parameters = grid_search.best_estimator_.get_params()
# for param_name in sorted(parametersSVC.keys()):
#     print("\t%s: %r" % (param_name, best_parameters[param_name]))

### cNEU

In [56]:
# cv = cross_validation.StratifiedKFold(range(neu.shape[0]), 2, shuffle=False, random_state=None)

# clas_fit_SVC = classifierSVC.fit(train_feat_neu.iloc[:,1], train_class_neu)
# predicted_SVC = classifierSVC.predict(test_feat_neu.iloc[:,1])
# scores_SVC = cross_validation.cross_val_score(classifierSVC, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
# print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
# print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_SVC, average='micro'))
# print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_SVC, labels=[0, 1], average='binary'))
# print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_SVC, labels=[0, 1], average='binary'))

In [66]:
clas_fit_SVC = classifierSVC.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_SVC = classifierSVC.predict(test_feat_neu.iloc[:,1])
scores_SVC = cross_validation.cross_val_score(classifierSVC, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_SVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_SVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_SVC, labels=[0, 1], average='binary'))


clas_fit_LSVC = classifierLinearSVC.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_LSVC = classifierLinearSVC.predict(test_feat_neu.iloc[:,1])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_LSVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_LSVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_LSVC, labels=[0, 1], average='binary'))


print("")

clas_fit_BNB = classifierBNB.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_BNB = classifierBNB.predict(test_feat_neu.iloc[:,1])
scores_BNB = cross_validation.cross_val_score(classifierBNB, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy bernuli NBC: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_BNB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_BNB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_BNB, labels=[0, 1], average='binary'))

clas_fit_NB = classifierNB.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_NB = classifierNB.predict(test_feat_neu.iloc[:,1])
scores_NB = cross_validation.cross_val_score(classifierNB, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy Multi-NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_NB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_NB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_NB, labels=[0, 1], average='binary'))

print("")

clas_fit_kNN = classifier_kNN.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_kNN = classifier_kNN.predict(test_feat_neu.iloc[:,1])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_kNN, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_kNN, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_kNN, labels=[0, 1], average='binary'))


print("")

clas_fit_ADC = classifierADC.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_ADC = classifierADC.predict(test_feat_neu.iloc[:,1])
scores_ADC = cross_validation.cross_val_score(classifierADC, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy AdaB: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_ADC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_ADC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_ADC, labels=[0, 1], average='binary'))



clas_fit_RF = classifierRF.fit(train_feat_neu.iloc[:,1], train_class_neu)
predicted_RF = classifierRF.predict(test_feat_neu.iloc[:,1])
scores_RF = cross_validation.cross_val_score(classifierRF, neu_X.iloc[:,1], neu_Y, cv = 10,  scoring='accuracy')
print("Accuracy RF: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_neu, predicted_RF, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_neu, predicted_RF, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_neu, predicted_RF, labels=[0, 1], average='binary'))

Accuracy rbf kernel SVC: 0.625 (+/- 0.00)
Precision:  0.687425860024
Recall:  0.0
F1:  0.0
Accuracy rbf kernel SVC: 0.628 (+/- 0.03)
Precision:  0.55101578895
Recall:  0.378955696203
F1:  0.427487728693

Accuracy rbf kernel SVC: 0.631 (+/- 0.01)
Precision:  0.61594315579
Recall:  0.0174050632911
F1:  0.0341085271318
Accuracy rbf kernel SVC: 0.634 (+/- 0.01)
Precision:  0.61594315579
Recall:  0.0174050632911
F1:  0.0341085271318

Accuracy rbf kernel SVC: 0.426 (+/- 0.03)
Precision:  0.627747677215
Recall:  0.808544303797
F1:  0.512537612839

Accuracy rbf kernel SVC: 0.628 (+/- 0.01)
Precision:  0.445617362108
Recall:  0.100474683544
F1:  0.164507772021
Accuracy rbf kernel SVC: 0.610 (+/- 0.02)
Precision:  0.479450855218
Recall:  0.197784810127
F1:  0.276701715551


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


#### VotingClassifier cNUE

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [12]:
# eclf = VotingClassifier(estimators=[
#         ("svc", clas_fit_SVC),
#         ("linear_svc", clas_fit_LSVC), 
#         ("bnb", clas_fit_BNB),
#         ("nb", clas_fit_NB),
#         ("kNN", clas_fit_kNN), 
#         ("rf", clas_fit_RF), 
#         ("adc", clas_fit_ADC)], voting='hard')


# for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
#     scores = cross_validation.cross_val_score(clf, neu["STATUS"], neu["cNEU"], cv=5, scoring='accuracy')
#     print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.625 (+/- 0.00) [SVC]
Accuracy: 0.555 (+/- 0.00) [LSVC]
Accuracy: 0.625 (+/- 0.00) [BNB]
Accuracy: 0.624 (+/- 0.00) [NB]
Accuracy: 0.421 (+/- 0.03) [kNN]
Accuracy: 0.598 (+/- 0.01) [Random Forest]
Accuracy: 0.607 (+/- 0.01) [ADC]
Accuracy: 0.623 (+/- 0.00) [Ensemble]


### cAGR


In [67]:
clas_fit_SVC = classifierSVC.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_SVC = classifierSVC.predict(test_feat_agr.iloc[:,1])
scores_SVC = cross_validation.cross_val_score(classifierSVC, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_SVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_SVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_SVC, labels=[0, 1], average='binary'))


clas_fit_LSVC = classifierLinearSVC.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_LSVC = classifierLinearSVC.predict(test_feat_agr.iloc[:,1])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_LSVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_LSVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_LSVC, labels=[0, 1], average='binary'))


print("")

clas_fit_BNB = classifierBNB.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_BNB = classifierBNB.predict(test_feat_agr.iloc[:,1])
scores_BNB = cross_validation.cross_val_score(classifierBNB, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy bernuli NBC: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_BNB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_BNB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_BNB, labels=[0, 1], average='binary'))

clas_fit_NB = classifierNB.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_NB = classifierNB.predict(test_feat_agr.iloc[:,1])
scores_NB = cross_validation.cross_val_score(classifierNB, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy Multi-NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_NB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_NB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_NB, labels=[0, 1], average='binary'))

print("")

clas_fit_kNN = classifier_kNN.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_kNN = classifier_kNN.predict(test_feat_agr.iloc[:,1])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_kNN, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_kNN, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_kNN, labels=[0, 1], average='binary'))


print("")

clas_fit_ADC = classifierADC.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_ADC = classifierADC.predict(test_feat_agr.iloc[:,1])
scores_ADC = cross_validation.cross_val_score(classifierADC, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy AdaB: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_ADC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_ADC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_ADC, labels=[0, 1], average='binary'))



clas_fit_RF = classifierRF.fit(train_feat_agr.iloc[:,1], train_class_agr)
predicted_RF = classifierRF.predict(test_feat_agr.iloc[:,1])
scores_RF = cross_validation.cross_val_score(classifierRF, agr_X.iloc[:,1], agr_Y, cv = 10,  scoring='accuracy')
print("Accuracy RF: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_agr, predicted_RF, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_agr, predicted_RF, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_agr, predicted_RF, labels=[0, 1], average='binary'))

Accuracy rbf kernel SVC: 0.531 (+/- 0.00)
Precision:  0.765569395018
Recall:  1.0
F1:  0.693782684486
Accuracy LinearSVC: 0.596 (+/- 0.02)
Precision:  0.716005283038
Recall:  0.632607481854
F1:  0.61811238407

Accuracy bernuli NBC: 0.585 (+/- 0.03)
Precision:  0.762189232181
Recall:  0.908989391401
F1:  0.698412698413
Accuracy Multi-NB: 0.595 (+/- 0.03)
Precision:  0.760705049227
Recall:  0.881630374093
F1:  0.697438162544

Accuracy kNN: 0.487 (+/- 0.02)
Precision:  0.557983677194
Recall:  0.104969290899
F1:  0.175536881419

Accuracy AdaB: 0.539 (+/- 0.03)
Precision:  0.745601326357
Recall:  0.898380792853
F1:  0.673644546787
Accuracy RF: 0.549 (+/- 0.03)
Precision:  0.694049152049
Recall:  0.569514237856
F1:  0.579545454545


#### VotingClassifier cAGR

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [14]:
# eclf = VotingClassifier(estimators=[
#         ("svc", clas_fit_SVC),
#         ("linear_svc", clas_fit_LSVC),
#         ("bnb", clas_fit_BNB),
#         ("nb", clas_fit_NB),
#         ("kNN", clas_fit_kNN), 
#         ("rf", clas_fit_RF), 
#         ("adc", clas_fit_ADC)], voting='hard')


# for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC", "BNB","NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
#     scores = cross_validation.cross_val_score(clf, agr["STATUS"], agr["cAGR"], cv=5, scoring='accuracy')
#     print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.531 (+/- 0.00) [SVC]
Accuracy: 0.513 (+/- 0.01) [LSVC]
Accuracy: 0.524 (+/- 0.02) [BNB]
Accuracy: 0.525 (+/- 0.01) [NB]
Accuracy: 0.472 (+/- 0.01) [kNN]
Accuracy: 0.506 (+/- 0.01) [Random Forest]
Accuracy: 0.512 (+/- 0.02) [ADC]
Accuracy: 0.522 (+/- 0.02) [Ensemble]


### cOPN

In [69]:
clas_fit_SVC = classifierSVC.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_SVC = classifierSVC.predict(test_feat_opn.iloc[:,1])
scores_SVC = cross_validation.cross_val_score(classifierSVC, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_SVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_SVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_SVC, labels=[0, 1], average='binary'))


clas_fit_LSVC = classifierLinearSVC.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_LSVC = classifierLinearSVC.predict(test_feat_opn.iloc[:,1])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_LSVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_LSVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_LSVC, labels=[0, 1], average='binary'))


print("")

clas_fit_BNB = classifierBNB.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_BNB = classifierBNB.predict(test_feat_opn.iloc[:,1])
scores_BNB = cross_validation.cross_val_score(classifierBNB, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy bernuli NBC: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_BNB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_BNB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_BNB, labels=[0, 1], average='binary'))

clas_fit_NB = classifierNB.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_NB = classifierNB.predict(test_feat_opn.iloc[:,1])
scores_NB = cross_validation.cross_val_score(classifierNB, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy Multi-NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_NB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_NB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_NB, labels=[0, 1], average='binary'))

print("")

clas_fit_kNN = classifier_kNN.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_kNN = classifier_kNN.predict(test_feat_opn.iloc[:,1])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_kNN, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_kNN, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_kNN, labels=[0, 1], average='binary'))


print("")

clas_fit_ADC = classifierADC.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_ADC = classifierADC.predict(test_feat_opn.iloc[:,1])
scores_ADC = cross_validation.cross_val_score(classifierADC, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy AdaB: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_ADC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_ADC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_ADC, labels=[0, 1], average='binary'))



clas_fit_RF = classifierRF.fit(train_feat_opn.iloc[:,1], train_class_opn)
predicted_RF = classifierRF.predict(test_feat_opn.iloc[:,1])
scores_RF = cross_validation.cross_val_score(classifierRF, opn_X.iloc[:,1], opn_Y, cv = 10,  scoring='accuracy')
print("Accuracy RF: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_opn, predicted_RF, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_opn, predicted_RF, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_opn, predicted_RF, labels=[0, 1], average='binary'))

Accuracy rbf kernel SVC: 0.743 (+/- 0.00)
Precision:  0.871589561091
Recall:  1.0
F1:  0.852670976523
Accuracy LinearSVC: 0.732 (+/- 0.03)
Precision:  0.874319677538
Recall:  0.896249002394
F1:  0.831389968536

Accuracy bernuli NBC: 0.747 (+/- 0.00)
Precision:  0.87287161173
Recall:  0.998802873105
F1:  0.85412045726
Accuracy Multi-NB: 0.746 (+/- 0.00)
Precision:  0.871920451172
Recall:  1.0
F1:  0.853106382979

Accuracy kNN: 0.670 (+/- 0.07)
Precision:  0.833460939208
Recall:  0.706703910615
F1:  0.724039247751

Accuracy AdaB: 0.740 (+/- 0.01)
Precision:  0.871812293797
Recall:  0.959297685555
F1:  0.844397611521
Accuracy RF: 0.726 (+/- 0.01)
Precision:  0.866237009533
Recall:  0.898643256185
F1:  0.822648401826


#### VotingClassifier cOPN

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [16]:
# eclf = VotingClassifier(estimators=[
#         ("svc", clas_fit_SVC),
#         ("linear_svc", clas_fit_LSVC),
#         ("bnb", clas_fit_BNB),
#         ("nb", clas_fit_NB),
#         ("kNN", clas_fit_kNN), 
#         ("rf", clas_fit_RF), 
#         ("adc", clas_fit_ADC)], voting='hard')


# for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC", "BNB","NB",'kNN', 'Random Forest', 'ADC', 'Ensemble']):
#     scores = cross_validation.cross_val_score(clf, opn["STATUS"], opn["cOPN"], cv=5, scoring='accuracy')
#     print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.743 (+/- 0.00) [SVC]
Accuracy: 0.682 (+/- 0.01) [LSVC]
Accuracy: 0.743 (+/- 0.00) [BNB]
Accuracy: 0.744 (+/- 0.00) [NB]
Accuracy: 0.672 (+/- 0.02) [kNN]
Accuracy: 0.705 (+/- 0.01) [Random Forest]
Accuracy: 0.707 (+/- 0.02) [ADC]
Accuracy: 0.743 (+/- 0.00) [Ensemble]


### cCON

In [68]:
clas_fit_SVC = classifierSVC.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_SVC = classifierSVC.predict(test_feat_con.iloc[:,1])
scores_SVC = cross_validation.cross_val_score(classifierSVC, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_SVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_SVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_SVC, labels=[0, 1], average='binary'))


clas_fit_LSVC = classifierLinearSVC.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_LSVC = classifierLinearSVC.predict(test_feat_con.iloc[:,1])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_LSVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_LSVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_LSVC, labels=[0, 1], average='binary'))


print("")

clas_fit_BNB = classifierBNB.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_BNB = classifierBNB.predict(test_feat_con.iloc[:,1])
scores_BNB = cross_validation.cross_val_score(classifierBNB, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy bernuli NBC: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_BNB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_BNB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_BNB, labels=[0, 1], average='binary'))

clas_fit_NB = classifierNB.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_NB = classifierNB.predict(test_feat_con.iloc[:,1])
scores_NB = cross_validation.cross_val_score(classifierNB, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy Multi-NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_NB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_NB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_NB, labels=[0, 1], average='binary'))

print("")

clas_fit_kNN = classifier_kNN.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_kNN = classifier_kNN.predict(test_feat_con.iloc[:,1])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_kNN, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_kNN, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_kNN, labels=[0, 1], average='binary'))


print("")

clas_fit_ADC = classifierADC.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_ADC = classifierADC.predict(test_feat_con.iloc[:,1])
scores_ADC = cross_validation.cross_val_score(classifierADC, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy AdaB: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_ADC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_ADC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_ADC, labels=[0, 1], average='binary'))



clas_fit_RF = classifierRF.fit(train_feat_con.iloc[:,1], train_class_con)
predicted_RF = classifierRF.predict(test_feat_con.iloc[:,1])
scores_RF = cross_validation.cross_val_score(classifierRF, con_X.iloc[:,1], con_Y, cv = 10,  scoring='accuracy')
print("Accuracy RF: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_con, predicted_RF, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_con, predicted_RF, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_con, predicted_RF, labels=[0, 1], average='binary'))

Accuracy rbf kernel SVC: 0.541 (+/- 0.00)
Precision:  0.729685646501
Recall:  0.0
F1:  0.0
Accuracy LinearSVC: 0.596 (+/- 0.02)
Precision:  0.64975311824
Recall:  0.520981278244
F1:  0.539078156313

Accuracy bernuli NBC: 0.601 (+/- 0.02)
Precision:  0.607496933616
Recall:  0.223369916075
F1:  0.33046800382
Accuracy Multi-NB: 0.604 (+/- 0.01)
Precision:  0.614616679188
Recall:  0.255003227889
F1:  0.363385464581

Accuracy kNN: 0.514 (+/- 0.04)
Precision:  0.704681744282
Recall:  0.903163331181
F1:  0.611050447696

Accuracy AdaB: 0.562 (+/- 0.02)
Precision:  0.56427053283
Recall:  0.287282117495
F1:  0.368530020704
Accuracy RF: 0.567 (+/- 0.04)
Precision:  0.606571873384
Recall:  0.414460942544
F1:  0.465048895328


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


#### VotingClassifier cCON

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [18]:
# eclf = VotingClassifier(estimators=[
#         ("svc", clas_fit_SVC),
#         ("linear_svc", clas_fit_LSVC), 
#         ("bnb", clas_fit_BNB),
#         ("nb", clas_fit_NB),
#         ("kNN", clas_fit_kNN), 
#         ("rf", clas_fit_RF), 
#         ("adc", clas_fit_ADC)], voting='hard')


# for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","BNB","NB",'kNN', 'Random Forest', 'ADC', 'Ensemble']):
#     scores = cross_validation.cross_val_score(clf, con["STATUS"], con["cCON"], cv=5, scoring='accuracy')
#     print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.541 (+/- 0.00) [SVC]
Accuracy: 0.528 (+/- 0.02) [LSVC]
Accuracy: 0.539 (+/- 0.02) [BNB]
Accuracy: 0.543 (+/- 0.02) [NB]
Accuracy: 0.512 (+/- 0.01) [kNN]
Accuracy: 0.529 (+/- 0.02) [Random Forest]
Accuracy: 0.533 (+/- 0.02) [ADC]
Accuracy: 0.542 (+/- 0.02) [Ensemble]


### cEXT

In [70]:
clas_fit_SVC = classifierSVC.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_SVC = classifierSVC.predict(test_feat_ext.iloc[:,1])
scores_SVC = cross_validation.cross_val_score(classifierSVC, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_SVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_SVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_SVC, labels=[0, 1], average='binary'))


clas_fit_LSVC = classifierLinearSVC.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_LSVC = classifierLinearSVC.predict(test_feat_ext.iloc[:,1])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_LSVC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_LSVC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_LSVC, labels=[0, 1], average='binary'))


print("")

clas_fit_BNB = classifierBNB.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_BNB = classifierBNB.predict(test_feat_ext.iloc[:,1])
scores_BNB = cross_validation.cross_val_score(classifierBNB, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy bernuli NBC: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_BNB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_BNB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_BNB, labels=[0, 1], average='binary'))

clas_fit_NB = classifierNB.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_NB = classifierNB.predict(test_feat_ext.iloc[:,1])
scores_NB = cross_validation.cross_val_score(classifierNB, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy Multi-NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_NB, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_NB, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_NB, labels=[0, 1], average='binary'))

print("")

clas_fit_kNN = classifier_kNN.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_kNN = classifier_kNN.predict(test_feat_ext.iloc[:,1])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_kNN, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_kNN, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_kNN, labels=[0, 1], average='binary'))


print("")

clas_fit_ADC = classifierADC.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_ADC = classifierADC.predict(test_feat_ext.iloc[:,1])
scores_ADC = cross_validation.cross_val_score(classifierADC, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy AdaB: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_ADC, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_ADC, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_ADC, labels=[0, 1], average='binary'))



clas_fit_RF = classifierRF.fit(train_feat_ext.iloc[:,1], train_class_ext)
predicted_RF = classifierRF.predict(test_feat_ext.iloc[:,1])
scores_RF = cross_validation.cross_val_score(classifierRF, ext_X.iloc[:,1], ext_Y, cv = 10,  scoring='accuracy')
print("Accuracy RF: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))
print("Precision: ", sk.metrics.average_precision_score(test_class_ext, predicted_RF, average='micro'))
print("Recall: ", sk.metrics.recall_score(test_class_ext, predicted_RF, labels=[0, 1], average='binary'))
print("F1: ", sk.metrics.f1_score(test_class_ext, predicted_RF, labels=[0, 1], average='binary'))


Accuracy rbf kernel SVC: 0.575 (+/- 0.00)
Precision:  0.7121886121
Recall:  0.0
F1:  0.0
Accuracy LinearSVC: 0.600 (+/- 0.03)
Precision:  0.603414830532
Recall:  0.45213137666
F1:  0.484644194757

Accuracy bernuli NBC: 0.605 (+/- 0.01)
Precision:  0.611361725991
Recall:  0.0936408106219
F1:  0.166356300435
Accuracy Multi-NB: 0.610 (+/- 0.01)
Precision:  0.578035701866
Recall:  0.115303983229
F1:  0.196545562835

Accuracy kNN: 0.528 (+/- 0.03)
Precision:  0.475125956234
Recall:  0.143955276031
F1:  0.217299578059

Accuracy AdaB: 0.576 (+/- 0.03)
Precision:  0.54159084921
Recall:  0.25786163522
F1:  0.342618384401
Accuracy RF: 0.574 (+/- 0.04)
Precision:  0.541389522014
Recall:  0.268343815514
F1:  0.350205198358


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


#### VotingClassifier cEXT

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [20]:
# eclf = VotingClassifier(estimators=[
#         ("svc", clas_fit_SVC),
#         ("linear_svc", clas_fit_LSVC), 
#         ("bnb", clas_fit_BNB),
#         ("nb", clas_fit_NB),
#         ("kNN", clas_fit_kNN), 
#         ("rf", clas_fit_RF), 
#         ("adc", clas_fit_ADC)], voting='hard')


# for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC", "BNB", "NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
#     scores = cross_validation.cross_val_score(clf, ext["STATUS"], ext["cEXT"], cv=5, scoring='accuracy')
#     print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.575 (+/- 0.00) [SVC]
Accuracy: 0.537 (+/- 0.01) [LSVC]
Accuracy: 0.571 (+/- 0.01) [BNB]
Accuracy: 0.568 (+/- 0.00) [NB]
Accuracy: 0.523 (+/- 0.01) [kNN]
Accuracy: 0.542 (+/- 0.01) [Random Forest]
Accuracy: 0.557 (+/- 0.01) [ADC]
Accuracy: 0.569 (+/- 0.01) [Ensemble]
